In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
# 基本ツール
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import matplotlib.ticker as ticker
import seaborn as sns
import time
import copy

from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  r2_score

# optuna

!pip install optuna
import optuna.integration.lightgbm as lgb
import optuna

In [ ]:
# GPU確認

!nvidia-smi


NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# 時間計測

def tic():
    # require to import time
    global start_time_tictoc
    start_time_tictoc = time.time()
    
    
def toc(tag = 'elapsed time'):
    if 'start_time_tictoc' in globals():
        print('{}: {:.5f} sec'.format(tag, time.time() - start_time_tictoc), end = '' )
    else:
        print('tic has not been called')
        
        




In [ ]:
raw_data   = pd.read_csv('/content/drive/MyDrive/kaggle/train.csv')
raw_data_  = pd.read_csv('/content/drive/MyDrive/kaggle/test.csv')
sample_submission  = pd.read_csv('/content/drive/MyDrive/kaggle/sample_submission.csv')
train_data = raw_data.copy()
test_data  = raw_data_.copy()

In [ ]:
X = train_data.iloc[:, 1:25]
y = train_data.iloc[:,25:]

X_test = test_data.iloc[:,1:25]



In [ ]:
#　ダミー化
X = pd.get_dummies(X, columns=['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9'])
X_test = pd.get_dummies(X_test, columns=['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9'])

X = X.drop('cat6_G',axis=1)


In [ ]:
# 正規化
X = (X - np.min(X)) / (np.max(X) - np.min(X))
X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test)) 

In [ ]:

"""
Optuna example that optimizes a regressor for table dataset using XGB tuner.
In this example, we optimize the validation rmse loss of continuous values .

"""

'\nOptuna example that optimizes a regressor for table dataset using XGB tuner.\nIn this example, we optimize the validation rmse loss of continuous values .\n\n'

In [ ]:
# 交差検証のためのtrainデータとtestデータの作成

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.20, random_state=0)




In [ ]:
def objective(trial):
    params = {
        'random_state': 1,
        'n_estimators': trial.suggest_categorical('n_estimators', [10000]),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.001,3),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 10),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7, 0.8,0.9,1]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.1,0.2,0.3,0.4]),
    }
    model = XGBRegressor(**params) 
    model.fit(train_x, train_y, eval_set=[(test_x,test_y)], early_stopping_rounds=100, verbose=0)
    y_pred = model.predict(test_x)
    rmse = mean_squared_error(test_y, y_pred, squared=False)
    
    return rmse

In [ ]:
# 最適値を見つける

tic()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

toc()


print('Best parameters:', study.best_trial.params)
print('Best RMSE:', study.best_trial.value)

NameError: ignored

In [ ]:
study.best_params

{'colsample_bytree': 0.3,
 'gamma': 4.146201121987952,
 'learning_rate': 0.027073125290181854,
 'max_depth': 8,
 'n_estimators': 1000,
 'reg_alpha': 9.917514467217439,
 'reg_lambda': 9.848858122110705,
 'subsample': 0.9}

In [ ]:
params = study.best_params
params['random_state'] = 0
params['n_estimators'] = 10000

In [ ]:
# 予測

model = XGBRegressor(**params)

model.fit(train_x, train_y,
           eval_set =[(train_x, train_y), (test_x, test_y)],
           eval_metric="rmse", verbose=200, early_stopping_rounds=100
           )

    
pred = model.predict(X_test)

[02:44:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:6.60475	validation_1-rmse:6.6055
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[200]	validation_0-rmse:0.840227	validation_1-rmse:0.845821
[400]	validation_0-rmse:0.834908	validation_1-rmse:0.843541
[600]	validation_0-rmse:0.832586	validation_1-rmse:0.843021
[800]	validation_0-rmse:0.831	validation_1-rmse:0.842825
[1000]	validation_0-rmse:0.829765	validation_1-rmse:0.842729
Stopping. Best iteration:
[1097]	validation_0-rmse:0.829277	validation_1-rmse:0.842697



In [ ]:
pred

array([7.5810595, 7.7833705, 7.672191 , ..., 7.5589004, 7.462643 ,
       7.2992177], dtype=float32)

In [ ]:
# 提出

sample_submission['target'] = pred

from google.colab import files
sample_submission.to_csv("output.csv")
files.download('output.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>